In [32]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from scipy.signal import convolve2d

In [2]:
horizontal_kernel = np.array([[ 1, 1, 1, 1]])
vertical_kernel = np.transpose(horizontal_kernel)
diag1_kernel = np.eye(4, dtype=np.uint8)
diag2_kernel = np.fliplr(diag1_kernel)
detection_kernels = [horizontal_kernel, vertical_kernel, diag1_kernel, diag2_kernel]

In [3]:
board = np.array([
    [0,0,1,0],
    [0,2,1,0],
    [0,2,1,0],
    [0,2,1,0],
])

In [4]:
def winning_move(board, player):
    for kernel in detection_kernels:
        if (convolve2d(board == player, kernel, mode="valid") == 4).any():
            return True
    return False

In [5]:
winning_move(board, 2)

False

# Balanced Random

In [17]:
game_history = []

def human_move(board):
    human_key = 1
    human_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(human_targets) == 0:
        #print('no plays left for human')
        raise ValueError("stale")
    human_col_choice = random.choice(human_targets) # random placement verticaly based on top row
    human_row_land = np.argmax(np.argwhere(board[:,human_col_choice] == 0)) # Lowest possible placement possible
    board[human_row_land, human_col_choice] = human_key
    return board

def robot_move(board):
    robot_key = 2
    robot_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(robot_targets) == 0:
        #print('no plays left for robot')
        raise ValueError("stale")
    robot_col_choice = random.choice(robot_targets) # random placement verticaly based on top row
    robot_row_land = np.argmax(np.argwhere(board[:,robot_col_choice] == 0)) # Lowest possible placement possible
    board[robot_row_land, robot_col_choice] = robot_key
    return board   

for i in range(0, 1000):
    board = np.array([
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
    ])
    players = [1, 2]
    first_player = random.choice(players)
    while True:
        try:
            if first_player == 1:
                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break

                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break
            elif first_player == 2:
                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break  

                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break
        except ValueError as e:
            game_history.append('Stale')
            break

df = pd.DataFrame(data=game_history, columns=['result'])
df['tick'] = 1
df.groupby('result')['tick'].sum()

result
Human Win    508
Robot Win    490
Stale          2
Name: tick, dtype: int64

# Precheck for winning Play

In [18]:
game_history = []

def human_move(board):
    human_key = 1
    human_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(human_targets) == 0:
        #print('no plays left for human')
        raise ValueError("stale")

    sim_win = False
    sim_win_target = 0
    for target in human_targets:
        board_copy = board.copy()
        human_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
        board_copy[human_row_land, target] = human_key
        sim_human_win = winning_move(board_copy, 1)
        if sim_human_win:
            sim_win = True
            sim_win_target = target
            break
    if sim_win:
        human_row_land = np.argmax(np.argwhere(board[:,sim_win_target] == 0)) # Lowest possible placement possible
        board[human_row_land, sim_win_target] = human_key
    else:
        human_col_choice = random.choice(human_targets) # random placement verticaly based on top row
        human_row_land = np.argmax(np.argwhere(board[:,human_col_choice] == 0)) # Lowest possible placement possible
        board[human_row_land, human_col_choice] = human_key
    return board

def robot_move(board):
    robot_key = 2
    robot_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(robot_targets) == 0:
        #print('no plays left for robot')
        raise ValueError("stale")
    robot_col_choice = random.choice(robot_targets) # random placement verticaly based on top row
    robot_row_land = np.argmax(np.argwhere(board[:,robot_col_choice] == 0)) # Lowest possible placement possible
    board[robot_row_land, robot_col_choice] = robot_key
    return board   

for i in range(0, 1000):
    board = np.array([
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
    ])
    players = [1, 2]
    first_player = random.choice(players)
    while True:
        try:
            if first_player == 1:
                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break

                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break
            elif first_player == 2:
                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break  

                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break
        except ValueError as e:
            game_history.append('Stale')
            break

df = pd.DataFrame(data=game_history, columns=['result'])
df['tick'] = 1
df.groupby('result')['tick'].sum()

result
Human Win    763
Robot Win    236
Stale          1
Name: tick, dtype: int64

# Precheck for Winning and Blocking Play

In [19]:
game_history = []
robot_key = 2
human_key = 1

def human_move(board):
    human_key = 1
    human_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(human_targets) == 0:
        #print('no plays left for human')
        raise ValueError("stale")

    sim_win = False
    sim_win_target = 0
    for target in human_targets:
        board_copy = board.copy()
        human_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
        board_copy[human_row_land, target] = human_key
        sim_human_win = winning_move(board_copy, 1)
        if sim_human_win:
            sim_win = True
            sim_win_target = target
            break
    if sim_win:
        human_row_land = np.argmax(np.argwhere(board[:,sim_win_target] == 0)) # Lowest possible placement possible
        board[human_row_land, sim_win_target] = human_key
    else:
        
        sim_loss = False
        sim_loss_target = 0
        
        enemy_targets = human_targets #renaming for simplicity
        
        for target in enemy_targets:
            board_copy = board.copy()
            enemy_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
            board_copy[enemy_row_land, target] = robot_key
            sim_human_loss = winning_move(board_copy, robot_key)
            if sim_human_loss:
                sim_loss = True
                sim_loss_target = target
                break
        if sim_loss:
            human_row_land = np.argmax(np.argwhere(board[:,sim_loss_target] == 0)) # Lowest possible placement possible
            board[human_row_land, sim_loss_target] = human_key
        else:
            human_col_choice = random.choice(human_targets) # random placement verticaly based on top row
            human_row_land = np.argmax(np.argwhere(board[:,human_col_choice] == 0)) # Lowest possible placement possible
            board[human_row_land, human_col_choice] = human_key
    return board

def robot_move(board):
    robot_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(robot_targets) == 0:
        #print('no plays left for robot')
        raise ValueError("stale")
    robot_col_choice = random.choice(robot_targets) # random placement verticaly based on top row
    robot_row_land = np.argmax(np.argwhere(board[:,robot_col_choice] == 0)) # Lowest possible placement possible
    board[robot_row_land, robot_col_choice] = robot_key
    return board   

for i in range(0, 1000):
    board = np.array([
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
    ])
    players = [1, 2]
    first_player = random.choice(players)
    while True:
        try:
            if first_player == 1:
                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break

                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break
            elif first_player == 2:
                board = robot_move(board)
                robot_win = winning_move(board, 2)
                if robot_win:
                    #print('robot win')
                    game_history.append('Robot Win')
                    break  

                board = human_move(board)
                human_win = winning_move(board, 1)
                if human_win:
                    #print('human win')
                    game_history.append('Human Win')
                    break
        except ValueError as e:
            game_history.append('Stale')
            break

df = pd.DataFrame(data=game_history, columns=['result'])
df['tick'] = 1
df.groupby('result')['tick'].sum()

result
Human Win    963
Robot Win     37
Name: tick, dtype: int64

# Convert to agnostic

In [23]:
game_history = []
robot_key = 2
human_key = 1

def smart_move(board, me, enemey):
    targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(targets) == 0:
        raise ValueError("stale")

    sim_win = False
    sim_win_target = 0
    for target in targets:
        board_copy = board.copy()
        me_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
        board_copy[me_row_land, target] = me
        sim_me_win = winning_move(board_copy, me)
        if sim_me_win:
            sim_win = True
            sim_win_target = target
            break
    if sim_win:
        me_row_land = np.argmax(np.argwhere(board[:,sim_win_target] == 0)) # Lowest possible placement possible
        board[me_row_land, sim_win_target] = me
    else:
        sim_loss = False
        sim_loss_target = 0
        
        for target in targets:
            board_copy = board.copy()
            enemy_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
            board_copy[enemy_row_land, target] = enemey
            sim_me_loss = winning_move(board_copy, enemey)
            if sim_me_loss:
                sim_loss = True
                sim_loss_target = target
                break
        if sim_loss:
            me_row_land = np.argmax(np.argwhere(board[:,sim_loss_target] == 0)) # Lowest possible placement possible
            board[me_row_land, sim_loss_target] = me
        else:
            me_col_choice = random.choice(targets) # random placement verticaly based on top row
            me_row_land = np.argmax(np.argwhere(board[:,me_col_choice] == 0)) # Lowest possible placement possible
            board[me_row_land, me_col_choice] = me
    return board

def random_move(board, me, enemy):
    me_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(me_targets) == 0:
        raise ValueError("stale")
    me_col_choice = random.choice(me_targets) # random placement verticaly based on top row
    me_row_land = np.argmax(np.argwhere(board[:,me_col_choice] == 0)) # Lowest possible placement possible
    board[me_row_land, me_col_choice] = me
    return board   

for i in range(0, 100):
    board = np.array([
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
    ])
    players = [1, 2]
    first_player = random.choice(players)
    while True:
        try:
            if first_player == 1:
                board = smart_move(board, human_key, robot_key)
                human_win = winning_move(board, 1)
                if human_win:
                    game_history.append('Human Win')
                    break

                board = random_move(board, robot_key, human_key)
                robot_win = winning_move(board, 2)
                if robot_win:
                    game_history.append('Robot Win')
                    break
            elif first_player == 2:
                board = random_move(board, robot_key, human_key)
                robot_win = winning_move(board, 2)
                if robot_win:
                    game_history.append('Robot Win')
                    break  

                board = smart_move(board, human_key, robot_key)
                human_win = winning_move(board, 1)
                if human_win:
                    game_history.append('Human Win')
                    break
        except ValueError as e:
            game_history.append('Stale')
            break

df = pd.DataFrame(data=game_history, columns=['result'])
df['tick'] = 1
df.groupby('result')['tick'].sum()

result
Human Win    99
Robot Win     1
Name: tick, dtype: int64

# make smart look two tiers into the futre for failure

In [44]:
game_history = []
robot_key = 2
human_key = 1

def smart_move(board, me, enemey, recurse_count = 0):
    targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(targets) == 0:
        raise ValueError("stale")

    sim_win = False
    sim_win_target = 0
    for target in targets:
        board_copy = board.copy()
        me_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
        board_copy[me_row_land, target] = me
        sim_me_win = winning_move(board_copy, me)
        if sim_me_win:
            sim_win = True
            sim_win_target = target
            break
    if sim_win:
        me_row_land = np.argmax(np.argwhere(board[:,sim_win_target] == 0)) # Lowest possible placement possible
        board[me_row_land, sim_win_target] = me
    else:
        sim_loss = False
        sim_loss_target = 0
        
        for target in targets:
            board_copy = board.copy()
            enemy_row_land = np.argmax(np.argwhere(board[:,target] == 0)) # Lowest possible placement possible
            board_copy[enemy_row_land, target] = enemey
            sim_me_loss = winning_move(board_copy, enemey)
            if sim_me_loss:
                sim_loss = True
                sim_loss_target = target
                break
        if sim_loss:
            me_row_land = np.argmax(np.argwhere(board[:,sim_loss_target] == 0)) # Lowest possible placement possible
            board[me_row_land, sim_loss_target] = me
        else:
            perform_rand_move = False
            if len(targets) > 1 & recurse_count == 0:
                ## recurse this 1x times
                possible_valid_target = None
                for target in targets:
                    recurse_board_1 = board.copy()
                    me_col_choice = target
                    me_row_land = np.argmax(np.argwhere(recurse_board_1[:,me_col_choice] == 0)) # Lowest possible placement possible
                    recurse_board_1[me_row_land, me_col_choice] = me
                    # assume enemy plays smart
                    recurse_count = recurse_count + 1
                    recurse_board_2 = smart_move(recurse_board_1, enemey, me, recurse_count)
                    enemy_win = winning_move(recurse_board_2, enemey)
                    if not enemy_win:
                        possible_valid_target = target
                        break
                if possible_valid_target != None:
                    me_col_choice = possible_valid_target # random placement verticaly based on top row
                    me_row_land = np.argmax(np.argwhere(board[:,me_col_choice] == 0)) # Lowest possible placement possible
                    board[me_row_land, me_col_choice] = me
                else:
                    perform_rand_move = True
            else:
                perform_rand_move = True
                
            if perform_rand_move:
                me_col_choice = random.choice(targets) # random placement verticaly based on top row
                me_row_land = np.argmax(np.argwhere(board[:,me_col_choice] == 0)) # Lowest possible placement possible
                board[me_row_land, me_col_choice] = me
    return board

def random_move(board, me, enemy):
    me_targets = np.argwhere(board[:1] == 0)[:,1].tolist()
    if len(me_targets) == 0:
        raise ValueError("stale")
    me_col_choice = random.choice(me_targets) # random placement verticaly based on top row
    me_row_land = np.argmax(np.argwhere(board[:,me_col_choice] == 0)) # Lowest possible placement possible
    board[me_row_land, me_col_choice] = me
    return board   

for i in tqdm(range(0, 1000)):
    board = np.array([
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0],
    ])
    players = [human_key, robot_key]
    first_player = random.choice(players)
    while True:
        try:
            if first_player == 1:
                board = smart_move(board, human_key, robot_key)
                human_win = winning_move(board, 1)
                if human_win:
                    game_history.append('Human Win')
                    break

                board = random_move(board, robot_key, human_key)
                robot_win = winning_move(board, 2)
                if robot_win:
                    game_history.append('Robot Win')
                    break
            elif first_player == 2:
                board = random_move(board, robot_key, human_key)
                robot_win = winning_move(board, 2)
                if robot_win:
                    game_history.append('Robot Win')
                    break  

                board = smart_move(board, human_key, robot_key)
                human_win = winning_move(board, 1)
                if human_win:
                    game_history.append('Human Win')
                    break
        except ValueError as e:
            game_history.append('Stale')
            break

df = pd.DataFrame(data=game_history, columns=['result'])
df['tick'] = 1
df.groupby('result')['tick'].sum()

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 186.43it/s]


result
Human Win    969
Robot Win     30
Stale          1
Name: tick, dtype: int64

# Future Sim Recursion

In [10]:
board.shape[0]

6

In [11]:
np.isin(board[:1], 0)

array([[ True,  True,  True,  True,  True,  True,  True]])

In [12]:
board[:,1]

array([0, 0, 0, 0, 2, 2])

In [13]:
np.argmax(np.argwhere(board[:,0] == 0))

1

In [14]:
min(board[:,2])

0

In [15]:
random.choice(np.argwhere(board[:1] == 0)[:,1].tolist())

6